In [1]:
import pandas as pd
import sklearn
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [2]:
appRecord = pd.read_csv("application_record.csv");
creditRecord = pd.read_csv("credit_record.csv");

In [3]:
creditRecord.insert(1,"STATUSINT",creditRecord['STATUS']);

In [4]:
def statusToInt(status):
    if status == "X": return 0
    elif status == "C": return 0
    elif status == "0": return 1
    elif status == "1": return 2
    elif status == "2": return 3
    elif status == "3": return 4
    elif status == "4": return 5
    else: return 6

In [5]:
creditRecord["STATUSINT"]=creditRecord["STATUS"].map(statusToInt)

In [6]:
creditRecord.insert(1,"TOTALMONTH",creditRecord['MONTHS_BALANCE']);

In [7]:
creditRecord["TOTALMONTH"]=creditRecord["MONTHS_BALANCE"].map(lambda x: -1*x);

In [8]:
maximum = creditRecord.groupby("ID").max()

In [9]:
average = creditRecord.groupby("ID").mean()

In [10]:
del maximum['STATUSINT']
del maximum['MONTHS_BALANCE']
del maximum['STATUS']
maximum.head()

,TOTALMONTH
ID,
5001711,3
5001712,18
5001713,21
5001714,14
5001715,59


In [11]:
customData = data = pd.merge(
    maximum,
    average,
    on="ID",
    how="inner"
)
data = pd.merge(
    appRecord,
    customData,
    on="ID",
    how="inner"
)


In [12]:
# Everything before the last column and the first column is a feature for a crude model. 
X = data.iloc[:data.size,1:19];
X.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,TOTALMONTH_x
0,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,15
1,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,14
2,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,29
3,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,4
4,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,26


In [13]:
y = data.iloc[:data.size,19:20];
y.head()

,TOTALMONTH_y
0,7.5
1,7.0
2,14.5
3,2.0
4,24.0


In [18]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
label_encoder = LabelEncoder();
X['CODE_GENDER'] = label_encoder.fit_transform(X['CODE_GENDER']);
X['FLAG_OWN_CAR'] = label_encoder.fit_transform(X['FLAG_OWN_CAR']);
X['FLAG_OWN_REALTY'] = label_encoder.fit_transform(X['FLAG_OWN_REALTY']);
X['NAME_INCOME_TYPE'] = label_encoder.fit_transform(X['NAME_INCOME_TYPE']);
X['NAME_EDUCATION_TYPE'] = label_encoder.fit_transform(X['NAME_EDUCATION_TYPE']);
X['NAME_FAMILY_STATUS'] = label_encoder.fit_transform(X['NAME_FAMILY_STATUS']);
X['NAME_HOUSING_TYPE'] = label_encoder.fit_transform(X['NAME_HOUSING_TYPE']);
"""
Some people don't have occupation information. This will throw a TypeError for the label_encoder. Let's fix this. 
"""
def jobEncodingPrep(status):
    #print(status)
    if pd.isna(status) : return "No Job Info"
    else: return status
X["OCCUPATION_TYPE"]=X["OCCUPATION_TYPE"].map(jobEncodingPrep)
#print(X)

# This is a dumb encoding, as occupation should be one-hot encoded. 
X['OCCUPATION_TYPE'] = label_encoder.fit_transform(X['OCCUPATION_TYPE']);

In [26]:
from sklearn.ensemble import RandomForestRegressor
# as a test, make a forest model with just a single feature. 
clf = RandomForestRegressor();
clf.fit(X, np.ravel(y))
print(clf.score(X,np.ravel(y)));

0.9689873625239425


In [37]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [41]:
kf = KFold(n_splits = 10)
kf.get_n_splits(X,y)

10

In [ ]:
mse = []
train_scores = []
test_scores = []

for train_index,test_index in kf.split(X):
    X_train,X_test = X.iloc[train_index], X.iloc[test_index]
    y_train,y_test = y.iloc[train_index], y.iloc[test_index]
    
    clfs = RandomForestRegressor()
    clfs.fit(X_train,np.ravel(y_train))
    y_pred = clfs.predict(X_test)
    mse.append(mean_squared_error(y_pred,y_test))
    train_scores.append(clfs.score(X_train,y_train))
    test_scores.append(clfs.score(X_test,y_test))
    

In [ ]:
print("MSEs: ", mse)
print("Average MSE: ", np.mean(mse))
print("Train scores: ", train_scores)
print("Averag train sconp.mean(train_scores))
print(test_scores)
print(np.mean(test_scores))

In [57]:
clfs.score(X_train,y_train)

0.9693531373543813